# Download and Process the IMDB Dataset

In [1]:
!pip install --quiet gdown

# 1. Download the zipped IMDB dataset from Drive
# this is the unsup part of https://ai.stanford.edu/~amaas/data/sentiment/

!gdown "https://drive.google.com/uc?id=1PjJ5cop0pT6tcEw9-ZUstVMujx-o-QTB" -O imdb_dataset.zip

# 2. Unzip the downloaded file
!unzip -q imdb_dataset.zip -d imdb_data


Downloading...
From (original): https://drive.google.com/uc?id=1PjJ5cop0pT6tcEw9-ZUstVMujx-o-QTB
From (redirected): https://drive.google.com/uc?id=1PjJ5cop0pT6tcEw9-ZUstVMujx-o-QTB&confirm=t&uuid=da15c8f3-989d-46da-88ba-902cd522450f
To: /content/imdb_dataset.zip
100% 44.7M/44.7M [00:01<00:00, 44.0MB/s]


In [2]:
import os
import re
import string
import random
from collections import defaultdict, Counter
import math
from math import log, exp


In [3]:

def load_imdb_unsup_sentences(folder_path):
    """
    Loads text files from the IMDB 'unsup' (unsupervised) folder.
    split text by newline, strips text, and returns a list of raw lines.
    replace <br /> tags with special token <nl> token.
    """
    all_sentences = []
    # Loop through all files in the directory
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)  # Get full path to file

        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read().replace("<br />", " <nl> ")  # Replace <br /> with <nl>
            lines = text.split("\n")  # Split text by newline
            all_sentences.extend([line.strip() for line in lines if line.strip()])  # Remove empty lines

    return all_sentences

def remove_punctuation(text):
    """
    Removes punctuation from the text,
    but keeps <nl> tokens intact.
    """
    text = re.sub(r"[^\w\s<nl>]", "", text)  # Remove all punctuation except <nl>
    return text

def build_vocabulary(sentences):
    """
    lower each sentence,
    Splits each sentence on whitespace, removes punctuation,
    and builds a set of unique tokens (vocabulary).
    """
    vocab = set()
    for sentence in sentences:
        sentence = sentence.lower()  # Convert to lowercase
        sentence = remove_punctuation(sentence)  # Remove punctuation
        tokens = sentence.split()  # Tokenize (split words)
        vocab.update(tokens)  # Add tokens to vocabulary

    return vocab

def tokinize(sentences, vocab, unknown="<UNK>"):
    """
    lower each sentence,
    Splits each sentence on whitespace, removes punctuation,
    and replaces tokens not in the vocabulary with unknowen token.
    Returns the list of tokenized sentences.
    """
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()  # Convert to lowercase
        sentence = remove_punctuation(sentence)  # Remove punctuation
        tokens = sentence.split()  # Tokenize (split words by spaces)

        # Replace unknown words with <UNK>
        tokens = [token if token in vocab else unknown for token in tokens]

        tokenized_sentences.append(tokens)  # Add to the list

    return tokenized_sentences

In [4]:
imdb_folder = "imdb_data/unsup"
sentences = load_imdb_unsup_sentences(imdb_folder)

print(f"Number of raw sentences loaded: {len(sentences)}")
print(f"Example (first 2 sentences):\n{sentences[:2]}")


Number of raw sentences loaded: 50000
Example (first 2 sentences):
['At the beginning of this film, there\'s a tight shot on Brooke Shields\' baby face: she\'s watching something with interest and we hear a woman moaning just in front of her. Since we all know what "Pretty Baby" is about, one is to assume the child is watching some sexual act with curiosity. Actually, it\'s just the opposite. This is writer-director Louis Malle\'s clever way of laughing at the viewer, saying "You have the dirty mind, not I." It\'s a very smart way to begin to the picture, but little else occupied my mind after it got going. Why would Keith Carradine\'s colorless older man want to marry a pubescent prostitute? Nobody here is saying, especially not Carradine (who has one sullen expression to express every emotion). The photography and background scoring are gorgeous, however the story and characters provide no passion, no emotion. The film is like a stylish painting, but one full of dullards. *1/2 from *

In [5]:
assert len(sentences) == 50000, "Expected 50,000 sentences from the unsup folder."

In [6]:
random.seed(42)

def split_data(sentences, test_split=0.1):
    """
      shuffle the sentences
      split them into train and test sets (first 1-test_split of the data is the training)
      return the train and test sets
    """
    random.shuffle(sentences)

    # Compute the split index
    split_index = int(len(sentences) * (1 - test_split))

        # Split into train and test sets
    train_sentences = sentences[:split_index]
    test_sentences = sentences[split_index:]

    return train_sentences, test_sentences


In [7]:
train_sentences, test_sentences = split_data(sentences)

print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of test sentences: {len(test_sentences)}")

Number of training sentences: 45000
Number of test sentences: 5000


In [8]:
assert len(train_sentences) == 45000, "Expected 45,000 sentences for training."
assert len(test_sentences) == 5000, "Expected 5,000 sentences for testing."


In [9]:
vocab = build_vocabulary(train_sentences)
tokinized_sentences = tokinize(train_sentences, vocab)

print(f"Vocabulary size: {len(vocab)}")
print(f"Example tokens from first sentence: {tokinized_sentences[0][:10] if tokinized_sentences else 'No tokens loaded'} ...")


Vocabulary size: 161218
Example tokens from first sentence: ['i', 'watched', 'this', 'movie', 'just', 'for', 'the', 'sake', 'of', 'a'] ...


In [10]:
# assert len(vocab) == 161292, "Expected a vocabulary size of 171,591." #skip for replication problems
assert len(tokinized_sentences) == 45000, "Expected tokenized sentences count to match raw sentences."

example = "I love Natural language processing, and i want to be a great engineer."
assert len(example) == 70, "Example sentence length (in characters) does not match the expected 70."

example_tokens = tokinize([example], vocab)[0]
assert len(example_tokens) == 13, "Token count for the example sentence does not match the expected 13."


In [58]:

def pad_sentence(tokens, n):
    """
    Pads a list of tokens with <s> at the start (n-1 times)
    and </s> at the end (once).
    For example, if n=3, you add 2 <s> tokens at the start.
    """
    padded = ["<s>"] * (n - 1) + tokens + ["</s>"]
    return padded

def build_ngram_counts(tokenized_sentences, n):
    """
    Builds n-gram counts and (n-1)-gram counts from the given tokenized sentences.
    Each sentence is padded with <s> and </s>.

    Args:
        tokenized_sentences: list of lists, where each sub-list is a tokenized sentence.
        n: the order of the n-gram (e.g., 2 for bigrams, 3 for trigrams).
        vocab: set of known words. If provided, you can choose to handle out-of-vocab tokens.

    Returns:
        ngram_counts: Counter of n-grams (tuples of length n).
        context_counts: Counter of (n-1)-gram contexts.
    """
    ngram_counts = Counter()
    context_counts = Counter()
    for sentence in tokenized_sentences:
        padded_sentence = pad_sentence(sentence, n)
        for i in range(len(padded_sentence) - n + 1):
            ngram = tuple(padded_sentence[i:i + n])
            context = tuple(padded_sentence[i:i + n - 1])  # (n-1)-gram context
            ngram_counts[ngram] += 1
            context_counts[context] += 1

    return ngram_counts, context_counts

def laplace_probability(ngram, ngram_counts, context_counts, vocab_size, alpha=0.1):
    """
    Computes the probability of an n-gram using Laplace (add-alpha) smoothing.

    P(w_i | w_{i-(n-1)}, ..., w_{i-1}) =
        (count(ngram) + alpha) / (count(context) + alpha * vocab_size)

    Args:
        ngram: tuple of tokens representing the n-gram
        ngram_counts: Counter of n-grams
        context_counts: Counter of (n-1)-gram contexts
        vocab_size: size of the vocabulary
        alpha: smoothing parameter (1.0 = add-1 smoothing)

    Returns:
        Probability of the given n-gram.
    """
    context = ngram[:-1]  # Extract (n-1)-gram context
    ngram_count = ngram_counts.get(ngram, 0)  # Get count of the n-gram
    context_count = context_counts.get(context, 0)  # Get count of the context

    prob = (ngram_count + alpha) / (context_count + alpha * vocab_size)
    return prob




In [66]:
n = 3
ngram_counts, context_counts = build_ngram_counts(tokinized_sentences, n=n)
print(f"Number of bigrams: {len(ngram_counts)}")
print(f"Number of contexts: {len(context_counts)}")


Number of bigrams: 6107483
Number of contexts: 2270622


In [61]:
from math import log, exp

def predict_next_token(
    context_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n=2,
    alpha=1.0,
    top_k=5
):
    """
    Given a list of context tokens, predict the next token using the n-gram model.
    Returns the top_k predictions as (token, probability).
    """

    candidates = []
    if len(context_tokens) < n - 1:
        return []  # Not enough context

    context = tuple(context_tokens[-(n - 1):])  # Get the last (n-1) tokens

    for word in vocab:  # Iterate over all possible next words
        ngram = context + (word,)  # Form the n-gram
        prob = laplace_probability(ngram, ngram_counts, context_counts, len(vocab), alpha)
        candidates.append((word, prob))

    # Sort candidates by probability in descending order and return top_k results
    candidates.sort(key=lambda x: x[1], reverse=True)

    return candidates[:top_k]


def generate_text_with_limit(
    start_tokens,
    ngram_counts,
    context_counts,
    vocab,
    n=2,
    alpha=1.0,
    max_length=20
):
    """
    Generates text from an n-gram model until it sees </s>
    or reaches a maximum total length (max_length).

    Args:
      start_tokens (list): initial context to begin generation
      ngram_counts (Counter): trained n-gram counts
      context_counts (Counter): trained (n-1)-gram counts
      vocab (set): the model vocabulary
      n (int): n-gram order, 2 for bigram, 3 for trigram, etc.
      alpha (float): Laplace smoothing parameter
      max_length (int): maximum number of tokens to generate (including start_tokens)

    Returns:
      A list of tokens representing the generated sequence.
    """
    generated = list(start_tokens)  # Initialize with start tokens

    while len(generated) < max_length:
        context = tuple(generated[-(n - 1):])  # Get last (n-1) tokens as context
        predictions = predict_next_token(context, ngram_counts, context_counts, vocab, n, alpha, top_k=5)

        if not predictions:  # If no predictions are found, stop generation
            break

        next_token = random.choices([token for token, _ in predictions], weights=[prob for _, prob in predictions])[0]
        #next_token = predictions[0][0]  # Select the token with the highest probability

        if next_token == "</s>":  # Stop if end-of-sequence token is generated
            break

        generated.append(next_token)
    return generated

context = ["i", "love","the","film"]
generated_seq = generate_text_with_limit(
    start_tokens=context,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab=vocab,
    n=4,
    alpha=1.0,
    max_length=128
)

print("Generated Sequence:", generated_seq)


Generated Sequence: ['i', 'love', 'the', 'film', 'friscos', 'friscos', 'occupiedlike', 'friscos', 'naturalistically', 'runner1982', 'naturalistically', 'naturalistically', 'naturalistically', 'occupiedlike', 'runner1982', 'daresay', 'naturalistically', 'runner1982', 'daresay', 'occupiedlike', 'friscos', 'naturalistically', 'runner1982', 'friscos', 'occupiedlike', 'friscos', 'naturalistically', 'runner1982', 'daresay', 'daresay', 'runner1982', 'runner1982', 'naturalistically', 'occupiedlike', 'occupiedlike', 'friscos', 'occupiedlike', 'friscos', 'runner1982', 'daresay', 'daresay', 'friscos', 'friscos', 'daresay', 'naturalistically', 'friscos', 'friscos', 'friscos', 'daresay', 'runner1982', 'daresay', 'naturalistically', 'daresay', 'runner1982', 'naturalistically', 'runner1982', 'naturalistically', 'naturalistically', 'naturalistically', 'friscos', 'naturalistically', 'friscos', 'daresay', 'occupiedlike', 'friscos', 'daresay', 'occupiedlike', 'runner1982', 'daresay', 'friscos', 'daresay'

In [80]:
def calculate_perplexity(
    tokenized_sentences,
    ngram_counts,
    context_counts,
    vocab_size,
    n,
    alpha
):
    """
    Calculates the perplexity of an n-gram model (with Laplace smoothing)
    on a list of tokenized sentences.

    Args:
      tokenized_sentences: List of lists of tokens.
      ngram_counts: Counter of n-grams.
      context_counts: Counter of (n-1)-grams.
      vocab_size: Size of the vocabulary.
      n: n-gram order.
      alpha: Laplace smoothing parameter.

    Returns:
      A float representing the perplexity on the given dataset.
    """

    log_prob_sum = 0.0
    num_tokens = 0  # Total number of tokens in the dataset

    for sentence in tokenized_sentences:
        sentence = ["<s>"] * (n - 1) + sentence + ["</s>"]  # Add start & end tokens

        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i : i + n])  # Extract n-gram
            context = tuple(sentence[i : i + n - 1])  # Extract (n-1)-gram context

            # Apply Laplace smoothing to compute probability
            ngram_count = ngram_counts.get(ngram, 0) + alpha
            context_count = context_counts.get(context, 0) + (alpha * vocab_size)
            #context_count = context_counts.get(context, 0) + (alpha * (vocab_size ** (n - 1)))

            prob = ngram_count / context_count  # Compute probability
            log_prob_sum += math.log(prob)  # Sum log probabilities
            num_tokens += 1


    perplexity = math.exp(-log_prob_sum / num_tokens) if num_tokens > 0 else float('inf')
    return perplexity

# **Analysis**
use different n and rerun the code and write down your analysis

In [81]:
n = 2
ngram_counts, context_counts = build_ngram_counts(tokinized_sentences, n=n)
print(f"Number of bigrams: {len(ngram_counts)}")
print(f"Number of contexts: {len(context_counts)}")

Number of bigrams: 2279086
Number of contexts: 161219


In [82]:
perplexity = calculate_perplexity(
    tokenized_sentences=tokinized_sentences,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab_size=len(vocab),
    n=2,
    alpha=0.1  # Laplace smoothing parameter
)
print(f"Perplexity: {perplexity}")

Perplexity: 741.6868175250855


In [64]:
tokenized_test_sentences = tokinize(test_sentences, vocab)

In [83]:
test_perplexity = calculate_perplexity(
    tokenized_sentences=tokenized_test_sentences,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab_size=len(vocab),
    n=2,
    alpha=0.1
)
print(f"Test Set Perplexity: {test_perplexity}")


Test Set Perplexity: 1170.1378997087822


In [84]:
n = 3
ngram_counts, context_counts = build_ngram_counts(tokinized_sentences, n=n)
print(f"Number of bigrams: {len(ngram_counts)}")
print(f"Number of contexts: {len(context_counts)}")

Number of bigrams: 6107483
Number of contexts: 2270622


In [85]:
perplexity = calculate_perplexity(
    tokenized_sentences=tokinized_sentences,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab_size=len(vocab),
    n=3,
    alpha=0.1  # Laplace smoothing parameter
)
print(f"Perplexity: {perplexity}")

Perplexity: 3833.4672536195835


In [86]:
test_perplexity = calculate_perplexity(
    tokenized_sentences=tokenized_test_sentences,
    ngram_counts=ngram_counts,
    context_counts=context_counts,
    vocab_size=len(vocab),
    n=3,
    alpha=0.1
)
print(f"Test Set Perplexity: {test_perplexity}")

Test Set Perplexity: 13949.357472607373


In [87]:
def test_pad_sentence():
    sentence = ["I", "love", "coding"]
    n = 3
    print("Original:", sentence)
    print("Padded:", pad_sentence(sentence, n))

test_pad_sentence()


Original: ['I', 'love', 'coding']
Padded: ['<s>', '<s>', 'I', 'love', 'coding', '</s>']


In [89]:
def test_count_ngrams():
    sentences = [["I", "love", "coding"], ["I", "love", "Python"]]
    n = 3
    ngram_counts, context_counts = build_ngram_counts(sentences, n)

    print("N-gram Counts:", ngram_counts)
    print("Context Counts:", context_counts)

test_count_ngrams()


N-gram Counts: Counter({('<s>', '<s>', 'I'): 2, ('<s>', 'I', 'love'): 2, ('I', 'love', 'coding'): 1, ('love', 'coding', '</s>'): 1, ('I', 'love', 'Python'): 1, ('love', 'Python', '</s>'): 1})
Context Counts: Counter({('<s>', '<s>'): 2, ('<s>', 'I'): 2, ('I', 'love'): 2, ('love', 'coding'): 1, ('love', 'Python'): 1})


In [90]:
def test_laplace_probability():
    ngram_counts = {('I', 'love', 'coding'): 1, ('I', 'love', 'Python'): 1}
    context_counts = {('I', 'love'): 2}
    vocab_size = 5  # Assume we have 5 unique words in the vocabulary
    ngram = ('I', 'love', 'coding')

    prob = laplace_probability(ngram, ngram_counts, context_counts, vocab_size)
    print(f"Laplace probability for {ngram}: {prob}")

test_laplace_probability()


Laplace probability for ('I', 'love', 'coding'): 0.44000000000000006


In [93]:
def test_perplexity():
    sentences = [["I", "love", "coding"], ["I", "love", "Python"]]
    n = 2
    vocab_size = 6  # Adjust based on actual unique words
    alpha = 1.0  # Laplace smoothing factor

    ngram_counts, context_counts = build_ngram_counts(sentences, n)
    perplexity = calculate_perplexity(sentences, ngram_counts, context_counts, vocab_size, n, alpha)

    print(f"Perplexity: {perplexity}")

test_perplexity()



Perplexity: 3.158758147025058


In [94]:
def test_perplexity():
    sentences = [["I", "love", "coding"], ["I", "love", "Python"]]

    for n in [2, 3]:  # Test for both bigrams and trigrams
        print(f"\nTesting for n={n}")

        ngram_counts, context_counts = build_ngram_counts(sentences, n)
        print("N-gram Counts:", ngram_counts)
        print("Context Counts:", context_counts)

        vocab_size = len(set(word for sent in sentences for word in sent))  # Compute vocab size
        alpha = 1.0

        perplexity = calculate_perplexity(sentences, ngram_counts, context_counts, vocab_size, n, alpha)
        print(f"Perplexity for n={n}: {perplexity}")

test_perplexity()



Testing for n=2
N-gram Counts: Counter({('<s>', 'I'): 2, ('I', 'love'): 2, ('love', 'coding'): 1, ('coding', '</s>'): 1, ('love', 'Python'): 1, ('Python', '</s>'): 1})
Context Counts: Counter({('<s>',): 2, ('I',): 2, ('love',): 2, ('coding',): 1, ('Python',): 1})
Perplexity for n=2: 2.340347319320716

Testing for n=3
N-gram Counts: Counter({('<s>', '<s>', 'I'): 2, ('<s>', 'I', 'love'): 2, ('I', 'love', 'coding'): 1, ('love', 'coding', '</s>'): 1, ('I', 'love', 'Python'): 1, ('love', 'Python', '</s>'): 1})
Context Counts: Counter({('<s>', '<s>'): 2, ('<s>', 'I'): 2, ('I', 'love'): 2, ('love', 'coding'): 1, ('love', 'Python'): 1})
Perplexity for n=3: 2.340347319320716
